In [1]:
from pathlib import Path
import os
from dotenv import load_dotenv
import kaggle
import zipfile

In [2]:
load_dotenv()

True

In [3]:
path = Path(f'{os.getenv("DATA_BASE_DIR")}/us-patent-phrase-to-phrase-matching')

In [ ]:
#!kaggle competitions download us-patent-phrase-to-phrase-matching -p {path}
# zipfile.ZipFile(f'{path}/{Path("us-patent-phrase-to-phrase-matching")}.zip').extractall(path)

In [9]:
!ls {path}

sample_submission.csv  train.csv
test.csv	       us-patent-phrase-to-phrase-matching.zip


In [4]:
import pandas as pd
from datasets import Dataset, DatasetDict

In [5]:
df = pd.read_csv(f"{path}/train.csv")

In [42]:
df.head()

,id,anchor,target,context,score
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75
2,36d72442aefd8232,abatement,active catalyst,A47,0.25
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00


In [43]:
df.describe(include="object")

,id,anchor,target,context
count,36473,36473,36473,36473
unique,36473,733,29340,106
top,8d135da0b55b8c88,component composite coating,composition,H01
freq,1,152,24,2186


In [44]:
df["score"].nunique()

5

In [45]:
df["score"].unique()

array([0.5 , 0.75, 0.25, 0.  , 1.  ])

The scores are values between 0 and 1, hence this can be modeled as a single class prediction problem

In [6]:
import torch
import os
import numpy as np
from transformers import AutoModelForSequenceClassification, AutoTokenizer

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [7]:
model_ckpt = "microsoft/deberta-v3-small"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt, cache_dir=os.getenv("HF_HUB_CACHE"))

/hps/software/users/pdbe/roshan/llm/llm_venv/lib/python3.11/site-packages/transformers/convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [49]:
tokenizer.sep_token

'[SEP]'

In [8]:
df["text"] = df["anchor"] + tokenizer.sep_token + df['target'] + tokenizer.sep_token + df['context']
df.rename(columns={"score":"label"}, inplace=True)

In [9]:
anchors = df.anchor.unique()
np.random.seed(42)
np.random.shuffle(anchors)

In [10]:
val_prop = 0.25
val_sz = int(len(anchors)*val_prop)
val_anchors = anchors[:val_sz]

In [11]:
train_df = df[~df['anchor'].isin(val_anchors)]
val_df = df[df['anchor'].isin(val_anchors)]

In [72]:
train_df.shape, val_df.shape

((27357, 6), (9116, 6))

In [83]:
train_df['label'].mean(), val_df['label'].mean()

(0.3623021530138539, 0.3613426941641071)

In [12]:
train_ds = Dataset.from_pandas(train_df)
val_ds = Dataset.from_pandas(val_df)

In [13]:
test_data = pd.read_csv(f"{path}/test.csv")
test_data["text"] = test_data["anchor"] + tokenizer.sep_token + test_data['target'] + tokenizer.sep_token + test_data['context']

In [14]:
test_ds = Dataset.from_pandas(test_data)

In [15]:
patents = DatasetDict({"train": train_ds,
                       "validation":val_ds,
                       "test": test_ds
                      })

In [93]:
patents

DatasetDict({
    train: Dataset({
        features: ['id', 'anchor', 'target', 'context', 'label', 'text', '__index_level_0__'],
        num_rows: 27357
    })
    validation: Dataset({
        features: ['id', 'anchor', 'target', 'context', 'label', 'text', '__index_level_0__'],
        num_rows: 9116
    })
    test: Dataset({
        features: ['id', 'anchor', 'target', 'context', 'text'],
        num_rows: 36
    })
})

In [94]:
patents['train'].features

{'id': Value(dtype='string', id=None),
 'anchor': Value(dtype='string', id=None),
 'target': Value(dtype='string', id=None),
 'context': Value(dtype='string', id=None),
 'label': Value(dtype='float64', id=None),
 'text': Value(dtype='string', id=None),
 '__index_level_0__': Value(dtype='int64', id=None)}

In [16]:
def tokenize(batch):
    return tokenizer(batch["text"], padding=True, truncation=True)

In [17]:
patents_encoded = patents.map(tokenize, batched=True, batch_size=None)

Map:   0%|          | 0/27357 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/9116 [00:00<?, ? examples/s]

Map:   0%|          | 0/36 [00:00<?, ? examples/s]

In [19]:
patents_encoded

DatasetDict({
    train: Dataset({
        features: ['id', 'anchor', 'target', 'context', 'score', 'text', 'label', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 27357
    })
    validation: Dataset({
        features: ['id', 'anchor', 'target', 'context', 'score', 'text', 'label', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 9116
    })
})

In [18]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [21]:
model = (AutoModelForSequenceClassification.from_pretrained(model_ckpt, num_labels=1, cache_dir=os.getenv("HF_HUB_CACHE")).to(device))

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-small and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [22]:
def compute_metrics(pred):
    return {'pearson': np.corrcoef(*pred)[0][1]}

In [23]:
from transformers import Trainer, TrainingArguments

In [24]:
import wandb

In [25]:
wandb.login()

wandb: ERROR Failed to detect the name of this notebook. You can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

  ········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /homes/roshan/.netrc
wandb: Currently logged in as: roshkjr (roshkjr-personal) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [26]:
%env WANDB_PROJECT="us-patent-phrase-to-phrase-matching"

env: WANDB_PROJECT="us-patent-phrase-to-phrase-matching"


In [27]:
batch_size = 128
lr = 8e-5
epochs = 4
wd = 0.01
logging_steps = len(patents_encoded["train"]) // batch_size
model_name = f"{model_ckpt}-finetuned-patents"
output_dir = f"{os.getenv('MODEL_BASE_DIR')}/model_name"

training_args = TrainingArguments(output_dir=output_dir,
                                  report_to = 'wandb', 
                                  num_train_epochs=epochs,
                                  learning_rate=lr,
                                  warmup_ratio=0.1,
                                  lr_scheduler_type='cosine',
                                  fp16=True,
                                  per_device_train_batch_size=batch_size,
                                  per_device_eval_batch_size=batch_size,
                                  weight_decay=wd, 
                                  eval_strategy="epoch",
                                  disable_tqdm=False,
                                  logging_steps=logging_steps,
                                  push_to_hub=False,
                                  log_level="error")

In [28]:
trainer = Trainer(model=model, args=training_args, compute_metrics=compute_metrics, train_dataset=patents_encoded["train"], eval_dataset=patents_encoded["validation"], processing_class=tokenizer)
trainer.train()

Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Epoch,Training Loss,Validation Loss,Pearson
1,0.045900,0.027606,0.794600
2,0.022200,0.024468,0.810178
3,0.015300,0.025127,0.813636
4,0.012000,0.024923,0.814069


TrainOutput(global_step=856, training_loss=0.023817308490799965, metrics={'train_runtime': 81.2506, 'train_samples_per_second': 1346.796, 'train_steps_per_second': 10.535, 'total_flos': 1217429129865960.0, 'train_loss': 0.023817308490799965, 'epoch': 4.0})

In [35]:
wandb.run()

In [66]:
tokenizer.model_input_names

['input_ids', 'token_type_ids', 'attention_mask']

In [86]:
patents_encoded.set_format("torch") #setting the format to torch so that we can use to(dvice) of torch

In [76]:
patents_encoded

DatasetDict({
    train: Dataset({
        features: ['id', 'anchor', 'target', 'context', 'label', 'text', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 27357
    })
    validation: Dataset({
        features: ['id', 'anchor', 'target', 'context', 'label', 'text', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 9116
    })
})

In [82]:
def get_output(batch):
    inputs = {k:v.to(device) for k,v in batch.items() if k in tokenizer.model_input_names}
    with torch.inference_mode():
        outputs = model(**inputs)
    return outputs

In [87]:
outputs = patents_encoded['validation'].map(get_output, batched=True)

Map:   0%|          | 0/9116 [00:00<?, ? examples/s]

In [88]:
outputs

Dataset({
    features: ['id', 'anchor', 'target', 'context', 'label', 'text', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask', 'logits'],
    num_rows: 9116
})

In [89]:
outputs['label']

tensor([0.5000, 0.2500, 0.2500,  ..., 0.0000, 0.5000, 0.7500])

In [90]:
outputs['logits']

tensor([[0.7075],
        [0.3140],
        [0.2856],
        ...,
        [0.0430],
        [0.5723],
        [0.6475]])